In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
def merge_dataframes_by_nearest_time(df0, df1, df2, time_column1, time_column2, max_time_diff=15):
    df0 = df0.rename(columns = {"SHOW":"CO"})
    df1 = pd.concat([df1, df0[["CO"]]],axis=1)
    
    df1 = df1.dropna()
    df2["created_at"] = pd.to_datetime(df2['created_at']) + pd.Timedelta(hours=1) # Change time from UTC to WAT
    
    """
    Merge two dataframes based on the nearest time within a maximum time difference.

    Args:
        df1 (pandas.DataFrame): First dataframe.
        df2 (pandas.DataFrame): Second dataframe.
        time_column1 (str): Name of the time column in df1.
        time_column2 (str): Name of the time column in df2.
        max_time_diff (float): Maximum time difference in seconds (default: 15).

    Returns:
        pandas.DataFrame: Merged dataframe.
    """
    # Convert the time columns to datetime format
    df1[time_column1] = pd.to_datetime(df1[time_column1]).dt.tz_localize(None)
    df2[time_column2] = pd.to_datetime(df2[time_column2]).dt.tz_localize(None)

    # Set the datetime column as the index for both dataframes
    df1.set_index(time_column1, inplace=True)
    df2.set_index(time_column2, inplace=True)

    # Create an empty dataframe to store the merged results
    merged_df = pd.DataFrame(columns=df1.columns)

    # Iterate over each row in df1
    for index, row in df1.iterrows():
        # Find the nearest timestamp within the maximum time difference
        nearest_timestamp = df2.index[np.abs((df2.index - index).total_seconds()) <= max_time_diff]
        
        # Check if there is a matching timestamp within the time difference
        if len(nearest_timestamp) > 0:
            # Choose the nearest timestamp with the smallest time difference
            nearest_timestamp = nearest_timestamp[np.abs((nearest_timestamp - index).total_seconds()).argmin()]
            
            # Create a new row with the merged values and append it to the merged dataframe
            merged_row = pd.concat([row, df2.loc[nearest_timestamp]])
            merged_df = merged_df.append(merged_row, ignore_index=True)

    # Reset the index of the merged dataframe
    merged_df.reset_index(drop=True, inplace=True)

    return merged_df

### Day 01 - Day 06 (Internal Storage)

In [5]:
alpha_dataset = pd.DataFrame()
for i in range(6):
    print("Processing ################# Day {} ############################".format(i+1))
    alpha = pd.read_csv("Day 0{}/alpha.csv".format(i+1))
    co2 =  pd.read_csv("Day 0{}/CO2.csv".format(i+1))
    co =  pd.read_csv("Day 0{}/CO.csv".format(i+1))

    merged_data = merge_dataframes_by_nearest_time(co, co2, alpha, "TIME","created_at", max_time_diff=10)
    merged_data = merged_data.drop(["ID","STATE", "elevation", "entry_id", "latitude","longitude", "status"], axis=1).rename(columns={'SHOW': 'CO2',"field1":"Temperature", "field2":"Humidity", "field3":"MQ7_analog", "field4":"MQ9_analog", "field5":"MG811_analog", "field6":"MQ135_analog", "field7":"MG811_Digital", "field8":"MQ7_Digital"})
    merged_data.tail()
    alpha_dataset = alpha_dataset.append(merged_data)

alpha_dataset = alpha_dataset.reset_index(drop=True)
alpha_dataset.to_csv("alpha.csv", index=False)
alpha_dataset.tail()

Processing ################# Day 1 ############################
Processing ################# Day 2 ############################
Processing ################# Day 3 ############################
Processing ################# Day 4 ############################
Processing ################# Day 5 ############################
Processing ################# Day 6 ############################


,CO2,CO,Temperature,Humidity,MQ7_analog,MQ9_analog,MG811_analog,MQ135_analog,MG811_Digital,MQ7_Digital
2693,603.0,0.0,33.37,54.62,3048.0,1306.0,4938.0,2445.0,0.0,1.0
2694,603.0,0.0,33.29,55.17,2983.0,1272.0,4919.0,2346.0,0.0,1.0
2695,603.0,0.0,33.28,54.62,3068.0,1309.0,4916.0,2419.0,0.0,1.0
2696,602.0,0.0,33.30,54.60,3138.0,1375.0,4978.0,2484.0,0.0,1.0
2697,602.0,0.0,33.33,54.71,3071.0,1318.0,4972.0,2435.0,0.0,1.0


In [6]:
beta_dataset = pd.DataFrame()
for i in range(6):
    print("Processing ################# Day {} ############################".format(i+1))
    beta = pd.read_csv("Day 0{}/Beta.csv".format(i+1))
    co2 =  pd.read_csv("Day 0{}/CO2.csv".format(i+1))
    co =  pd.read_csv("Day 0{}/CO.csv".format(i+1))

    merged_data = merge_dataframes_by_nearest_time(co, co2, beta, "TIME","created_at", max_time_diff=10)
    merged_data = merged_data.drop(["ID","STATE", "elevation", "entry_id", "latitude","longitude", "status"], axis=1).rename(columns={'SHOW': 'CO2',"field1":"Temperature", "field2":"Humidity", "field3":"MQ7_analog", "field4":"MQ9_analog", "field5":"MG811_analog", "field6":"MQ135_analog", "field7":"MG811_Digital", "field8":"MQ7_Digital"})
    merged_data.tail()
    beta_dataset = beta_dataset.append(merged_data)

beta_dataset = beta_dataset.reset_index(drop=True)
beta_dataset.to_csv("beta.csv", index=False)
beta_dataset.tail()

Processing ################# Day 1 ############################
Processing ################# Day 2 ############################
Processing ################# Day 3 ############################
Processing ################# Day 4 ############################
Processing ################# Day 5 ############################
Processing ################# Day 6 ############################


,CO2,CO,Temperature,Humidity,MQ7_analog,MQ9_analog,MG811_analog,MQ135_analog,MG811_Digital,MQ7_Digital
2796,599.0,0.0,29.05,67.85,4639.0,3947.0,2765.0,3660.0,0.0,1.0
2797,598.0,0.0,29.03,67.99,4639.0,3941.0,2764.0,3654.0,0.0,1.0
2798,599.0,0.0,29.01,68.66,4641.0,3945.0,2762.0,3661.0,0.0,1.0
2799,598.0,0.0,29.08,68.78,3402.0,4144.0,3098.0,3541.0,0.0,1.0
2800,598.0,0.0,29.06,68.45,3582.0,3973.0,2736.0,3588.0,0.0,1.0


In [7]:
charlie_dataset = pd.DataFrame()
for i in range(6):
    print("Processing ################# Day {} ############################".format(i+1))
    charlie = pd.read_csv("Day 0{}/Charlie.csv".format(i+1))
    co2 =  pd.read_csv("Day 0{}/CO2.csv".format(i+1))
    co =  pd.read_csv("Day 0{}/CO.csv".format(i+1))

    merged_data = merge_dataframes_by_nearest_time(co, co2, charlie, "TIME","created_at", max_time_diff=10)
    merged_data = merged_data.drop(["ID","STATE", "elevation", "entry_id", "latitude","longitude", "status"], axis=1).rename(columns={'SHOW': 'CO2',"field1":"Temperature", "field2":"Humidity", "field3":"MQ7_analog", "field4":"MQ9_analog", "field5":"MG811_analog", "field6":"MQ135_analog", "field7":"MG811_Digital", "field8":"MQ7_Digital"})
    merged_data.tail()
    charlie_dataset = beta_dataset.append(merged_data)

charlie_dataset = charlie_dataset.reset_index(drop=True)
charlie_dataset.to_csv("charlie.csv", index=False)
charlie_dataset.tail()

Processing ################# Day 1 ############################
Processing ################# Day 2 ############################
Processing ################# Day 3 ############################
Processing ################# Day 4 ############################
Processing ################# Day 5 ############################
Processing ################# Day 6 ############################


,CO2,CO,Temperature,Humidity,MQ7_analog,MQ9_analog,MG811_analog,MQ135_analog,MG811_Digital,MQ7_Digital
3026,599.0,0.0,30.26,64.21,4962.0,4746.0,4320.0,3749.0,0.0,1.0
3027,600.0,0.0,30.25,64.19,4956.0,4742.0,4319.0,3749.0,0.0,1.0
3028,597.0,0.0,30.26,64.28,4937.0,4721.0,4310.0,3739.0,0.0,1.0
3029,598.0,0.0,30.27,64.60,5066.0,4859.0,4486.0,3846.0,0.0,1.0
3030,598.0,0.0,30.28,64.33,4860.0,4620.0,4154.0,3603.0,0.0,1.0
